In [21]:
import numpy as np
import pandas as pd
import re
from androguard.core.bytecodes.apk import APK
from androguard.core.bytecodes.dvm import DalvikVMFormat
from androguard.misc import AnalyzeAPK

In [2]:
features = pd.read_csv("../dataset/dataset-features-categories.csv", names=["X", "Category"])
features.head()

,X,Category
0,transact,API call signature
1,onServiceConnected,API call signature
2,bindService,API call signature
3,attachInterface,API call signature
4,ServiceConnection,API call signature


In [3]:
features["Category"].value_counts()

Manifest Permission    113
API call signature      73
Intent                  23
Commands signature       6
B=Benign; S=Malware      1
Name: Category, dtype: int64

In [4]:
features[features["Category"] == "API call signature"]

,X,Category
0,transact,API call signature
1,onServiceConnected,API call signature
2,bindService,API call signature
3,attachInterface,API call signature
4,ServiceConnection,API call signature
...,...,...
167,ProcessBuilder,API call signature
171,Ljava.lang.Class.getResource,API call signature
172,defineClass,API call signature
180,findClass,API call signature


In [5]:
features[features["Category"] == "Intent"]

,X,Category
25,android.intent.action.BOOT_COMPLETED,Intent
53,android.intent.action.PACKAGE_REPLACED,Intent
55,android.intent.action.SEND_MULTIPLE,Intent
66,android.intent.action.TIME_SET,Intent
73,android.intent.action.PACKAGE_REMOVED,Intent
74,android.intent.action.TIMEZONE_CHANGED,Intent
81,android.intent.action.ACTION_POWER_DISCONNECTED,Intent
82,android.intent.action.PACKAGE_ADDED,Intent
97,android.intent.action.ACTION_SHUTDOWN,Intent
107,android.intent.action.PACKAGE_DATA_CLEARED,Intent


In [6]:
features[features["Category"] == "Commands signature"]

,X,Category
44,mount,Commands signature
79,chmod,Commands signature
96,remount,Commands signature
100,chown,Commands signature
141,/system/bin,Commands signature
149,/system/app,Commands signature


In [7]:
features.shape

(216, 2)

In [8]:
permissions_list = features[features["Category"] == "Manifest Permission"].X.unique()

In [9]:
permissions_list

array(['SEND_SMS', 'READ_PHONE_STATE', 'GET_ACCOUNTS', 'RECEIVE_SMS',
       'READ_SMS', 'USE_CREDENTIALS', 'MANAGE_ACCOUNTS', 'WRITE_SMS',
       'READ_SYNC_SETTINGS', 'AUTHENTICATE_ACCOUNTS',
       'WRITE_HISTORY_BOOKMARKS', 'INSTALL_PACKAGES', 'CAMERA',
       'WRITE_SYNC_SETTINGS', 'READ_HISTORY_BOOKMARKS', 'INTERNET',
       'RECORD_AUDIO', 'NFC', 'ACCESS_LOCATION_EXTRA_COMMANDS',
       'WRITE_APN_SETTINGS', 'BIND_REMOTEVIEWS', 'READ_PROFILE',
       'MODIFY_AUDIO_SETTINGS', 'READ_SYNC_STATS', 'BROADCAST_STICKY',
       'WAKE_LOCK', 'RECEIVE_BOOT_COMPLETED', 'RESTART_PACKAGES',
       'BLUETOOTH', 'READ_CALENDAR', 'READ_CALL_LOG',
       'SUBSCRIBED_FEEDS_WRITE', 'READ_EXTERNAL_STORAGE', 'VIBRATE',
       'ACCESS_NETWORK_STATE', 'SUBSCRIBED_FEEDS_READ',
       'CHANGE_WIFI_MULTICAST_STATE', 'WRITE_CALENDAR', 'MASTER_CLEAR',
       'UPDATE_DEVICE_STATS', 'WRITE_CALL_LOG', 'DELETE_PACKAGES',
       'GET_TASKS', 'GLOBAL_SEARCH', 'DELETE_CACHE_FILES',
       'WRITE_USER_DICTIONARY',

In [10]:
permissions_list.shape

(113,)

In [11]:
api_call_signatures = features[features["Category"] == "API call signature"].X.unique()

In [12]:
api_call_signatures

array(['transact', 'onServiceConnected', 'bindService', 'attachInterface',
       'ServiceConnection', 'android.os.Binder',
       'Ljava.lang.Class.getCanonicalName', 'Ljava.lang.Class.getMethods',
       'Ljava.lang.Class.cast', 'Ljava.net.URLDecoder',
       'android.content.pm.Signature', 'android.telephony.SmsManager',
       'getBinder', 'ClassLoader',
       'Landroid.content.Context.registerReceiver',
       'Ljava.lang.Class.getField',
       'Landroid.content.Context.unregisterReceiver',
       'Ljava.lang.Class.getDeclaredField', 'getCallingUid',
       'Ljavax.crypto.spec.SecretKeySpec',
       'android.content.pm.PackageInfo', 'KeySpec',
       'TelephonyManager.getLine1Number', 'DexClassLoader',
       'HttpGet.init', 'SecretKey', 'Ljava.lang.Class.getMethod',
       'System.loadLibrary', 'android.intent.action.SEND',
       'Ljavax.crypto.Cipher', 'android.telephony.gsm.SmsManager',
       'TelephonyManager.getSubscriberId', 'Runtime.getRuntime',
       'Ljava.lang.Objec

In [13]:
api_call_signatures.shape

(73,)

In [14]:
df = pd.read_csv("../dataset/drebin-215-dataset-5560malware-9476-benign.csv")

/tmp/ipykernel_8696/1373337510.py:1: DtypeWarning: Columns (92) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../dataset/drebin-215-dataset-5560malware-9476-benign.csv")


In [15]:
df.shape

(15036, 216)

In [43]:
apk_file_path = "/home/alper/İndirilenler/InsecureBankv2.apk"

In [44]:
a = APK(apk_file_path)
d = DalvikVMFormat(a.get_dex())

In [45]:
found_api_signatures = []

for method in d.get_methods():
    for api_call in api_call_signatures:
        if re.search(api_call, method.get_descriptor()):
            #print("[+]", perm, " found.")
            found_api_signatures.append(api_call)

In [46]:
set(found_api_signatures)

{'Binder',
 'ClassLoader',
 'HttpUriRequest',
 'IBinder',
 'ServiceConnection',
 'android.content.pm.PackageInfo',
 'android.os.IBinder'}

In [47]:
permissions = a.get_permissions()

In [50]:
found_permissions = []

for permission in permissions:
    permission = permission.split(".")[-1]
    if permission in permissions_list:
        found_permissions.append(permission)

In [51]:
found_permissions

['ACCESS_NETWORK_STATE',
 'ACCESS_COARSE_LOCATION',
 'READ_PROFILE',
 'WRITE_EXTERNAL_STORAGE',
 'GET_ACCOUNTS',
 'READ_CONTACTS',
 'INTERNET',
 'SEND_SMS',
 'USE_CREDENTIALS']

In [40]:
intents = features[features["Category"] == "Intent"].X.unique()

In [52]:
manifest = a.get_android_manifest_xml()
intent_filters = manifest.findall(".//intent-filter")
found_intents = []
for intent_filter in intent_filters:
    action_elements = intent_filter.findall(".//action")
    for action_element in action_elements:
        action_value = action_element.get("{http://schemas.android.com/apk/res/android}name")
        for intent in intents:
            if re.search(intent, action_value):
                print(action_value)

In [53]:
found_intents

[]

In [59]:
keywords = ["mount", "/system/bin", "/system/app", "chmod", "remount", "chown"]
found_keywords = []

for method in d.get_methods():
    for keyword in keywords:
        try:
            if re.search(keyword, method.get_code().get_instruction()):
                found_keywords.append(keyword)
        except:
            pass

In [60]:
found_keywords

[]